Functions:
- Load/Extract new file
-  

In [1]:
import pandas as pd
import os
from datetime import datetime

from openpyxl import Workbook
from openpyxl.styles import PatternFill
# from pandas import ExcelWrite


In [29]:
master_file_directory = 'landing_zone\2026\2025-06\Honda\2026_Honda_Accessories-06-2025.xlsx'

processes_files_storage_path ='C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/ready_to_upload'


# masterfile = pd.read_excel( f'..\landing_zone\2026\2025-06\Honda\2026_Honda_Accessories-06-2025.xlsx', engine='openpyxl')

# masterfile = pd.read_excel(f'C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/landing_zone/2026/2025-06/Honda/2026_Honda_Accessories-06-2025.xlsx', engine='openpyxl')


# file_name ='2025-Honda-Master-June25'

# master_file_path_25 = f'C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/landing_zone/2026/2025-06/Honda/{file_name}.xlsx'

common_columns = ["Description", "Part Number", "List Price", "Labour", "Price", "Comments"]




<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\paxm\AppData\Local\Temp\ipykernel_11796\3710805870.py:1: SyntaxWarning: invalid escape sequence '\H'
  master_file_directory = 'landing_zone\2026\2025-06\Honda\2026_Honda_Accessories-06-2025.xlsx'


In [4]:
master_file_path =f'C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/landing_zone/2026/2025-06/Honda/2026-Honda_Accessories-06_2025.xlsx'

In [5]:

# create a function that will read take in an excel file as an input, and find all the worskheets in the file and create a dataframe for each worksheet. This function will return a dictionary of dataframes with the worksheet names as the keys.
def read_excel_sheets(file_path):
    """
    Reads an Excel file and returns a dictionary of DataFrames for each sheet.
    
    :param file_path: Path to the Excel file.
    :return: Dictionary with sheet names as keys and DataFrames as values.
    """
    xls = pd.ExcelFile(file_path)
    sheets_dict = {sheet_name: pd.read_excel(xls, sheet_name=sheet_name, engine='openpyxl') for sheet_name in xls.sheet_names}

    # Optionally, you can close the Excel file after reading
    # first check if the file is open and the sheet_dict is not empty
    if sheets_dict and hasattr(xls, 'close'):
        xls.close()
    
    return sheets_dict, xls.sheet_names
 

In [7]:
# master_dict, sheets_names = read_excel_sheets(master_file_path_25)

In [8]:
# create a function to clean up the dataframes by, looping through each dataframe in the dictionary, check every column name and replace any \n with a space from the names. and make sure that if the column name has a space at the end, it is removed. and ensure that their no 2 spaces in the column names.    


def clean_dataframe_columns(df_dict):
    """
    Cleans the column names of DataFrames in a dictionary by replacing newline characters with spaces.
    
    :param df_dict: Dictionary of DataFrames.
    :return: Dictionary with cleaned DataFrames.
    """
    for key, df in df_dict.items():
        df.columns = [col.replace('\n', ' ') for col in df.columns]
        df.columns = [col.replace('  ', ' ') for col in df.columns]
        df.columns = [col.strip() for col in df.columns]
        df_dict[key] = df
    return df_dict
  

In [10]:
# master_df_dict =clean_dataframe_columns(master_dict)

In [11]:
# create a function that takes in a make, year, and model name and returns a vehicle model number. The function will check if the model name is in the model_mapping dictionary, if yes, it will return the model number, if not, it will return an empty string.

# Placeholder for database search logic

model_mapping = {
        'Honda': {'2026': 
                        {'CRV': 
                            {'LX': '2026-Honda-LX', 'Sport': '2026-Honda-Sport', 'Trailsport': '2026-Honda-Trailsport', 'EX-L': '2026-Honda-EX-L', 'Touring': '2026-Honda-Touring', 'Sport-Hybrid':'2026-Honda-Sport-Hybrid'}, 
                        'Civic': {'LX': '2026-Honda-LX', 'Sport': '2026-Honda-Sport'}
                        },
                  '2025': 
                        {'CRV': 
                            {'LX': '2025-Honda-LX', 'Sport': '2025-Honda-Sport', 'Trailsport': '2025-Honda-Trailsport', 'EX-L': '2025-Honda-EX-L', 'Touring': '2025-Honda-Touring', 'Sport-Hybrid':'2025-Honda-Sport-Hybrid'}, 
                        'Civic': {'LX': '2025-Honda-LX', 'Sport': '2025-Honda-Sport'}
                        },
                  '2024': 
                        {'CRV': 
                            {'LX': '2024-Honda-LX', 'Sport': '2024-Honda-Sport', 'Trailsport': '2024-Honda-Trailsport', 'EX-L': '2024-Honda-EX-L', 'Touring': '2024-Honda-Touring', 'Sport-Hybrid':'2024-Honda-Sport-Hybrid'}, 
                        'Civic': {'LX': '2024-Honda-LX', 'Sport': '2024-Honda-Sport'}
                        },
                   },
        'Toyota': {'2026': {'RAV4':{'RAV4': '2026-Toyota-RAV4'}, 'Camry': {'Camry': '2026-Toyota-Camry'}}}
    }



def get_vehicle_model_number(make, year, trim, model_name):
    """
    Returns the vehicle model number based on make, year, and model name.
    
    :param make: Make of the vehicle.
    :param year: Year of the vehicle.
    :param model_name: Name of the vehicle model.
    :param trim: Trim of the vehicle model.
    :return: Vehicle model number or empty string if not found.
    """
    model_num = model_mapping.get(make, {}).get(year, {}).get(model_name, {}).get(trim, '')

    if model_num == '':
        return False
    
    return model_num

In [12]:
get_vehicle_model_number(make='Honda', year='2024', trim='CRV', model_name='Sport-Hybrid')

False

In [13]:
#  takes in 1 Df and create dfs for each of the model trims of that model
#  Returns a dictionary with all the trim-dfs (values) under the model name as key

def create_unique_trim_dfs_by_model(loaded_df, make, year, model_name):

    # check if df has content

    if loaded_df is not None:
        unique_model_dfs_dict = {}

        for column in loaded_df.columns:
            # Find the specific model columns
            if column not in common_columns and pd.notnull(loaded_df[column]).any():
                new_df = loaded_df[[column] + common_columns].copy() # create df for each model ono the sheet
            

                # drop all records with N/A or empty 
                new_df = new_df[new_df[column].notnull() & (new_df[column] != '')]
                               
                # Add year column and assign value
                new_df.insert (0,"Year", year)

                trim_name = column  # Set trim_name

                # Fetching Vehicle model
                model_num = get_vehicle_model_number(make=make, year=year, trim=trim_name, model_name = model_name)
                

                new_df.rename(columns={column:'Model'}, inplace=True)

                if model_num != False:        
                    new_df['Model'] = model_num
                else:
                    new_df['Model_Temp'] = model_name + '-' + trim_name + '-' + year

                # Change column names to standard Rate Importer columns
                if 'FRT' in new_df.columns:
                    new_df.rename(columns={'FRT':'Hours'}, inplace=True)
                if 'List Price' in new_df.columns:
                    new_df.rename(columns={'List Price':'Cost'}, inplace=True)
                if 'Installed Price' in new_df.columns:
                    new_df.rename(columns={'Installed Price':'Price'}, inplace=True)
                if 'Part Number' in new_df.columns:
                    new_df.rename(columns={'Part Number':'Part'}, inplace=True)
                if 'Dealer Net' in new_df.columns:
                    new_df.drop(column={'Dealer Net'}, inplace= True)
                
                

                if not new_df.empty:
                    if "unnamed" not in column.lower():
                        unique_model_dfs_dict[column] = new_df
    
    return unique_model_dfs_dict

In [14]:
# Initiate the process to create dfs for each models 1 by 1

def create_all_unique_model_df(master_df_dict, make, year):
    all_model_df_dict = {}

    for model in master_df_dict.keys():
        clean_model_name = model.split('-')[0]
        all_model_df_dict[clean_model_name]= create_unique_trim_dfs_by_model(master_df_dict.get(model), make = make, year=year, model_name=clean_model_name) 
        
    return all_model_df_dict 
 

In [17]:
# res = create_all_unique_model_df(master_df_dict, make='Honda', year='2025')


# res.get("CRV").keys()

In [26]:
# res.get("CRV").get("Sport")

In [39]:
# create a function that takes in a dictionary of dataframes and saves it as an excel sheet with each dataframe as a separate sheet. The function will take in the dictionary of dataframes and a file path to save the excel file. The function will return nothing. The function will use the pandas ExcelWriter to write the dataframes to the excel file. The function will also check if the file path exists, if not, it will create the directory. The function will also check if the file already exists, if yes, it will overwrite the file. I want to modify this function, the input will be a dictionary of a dataframes, a directory path, the year, and make. Withe provided directory path, th goal is to find the country folder first, say 2026, if it doesn't exist create it, and then find the make, it doesn't exist create it based on the provided string, and then dave the fail in the format of "YYYY-Make_Accessories-UploadFile-timestamp with seconds.xlsx". The function will also ensure that the directory exists before saving the file. 

def save_dfs_to_excel(dict_all_dfs_by_model_ready, directory_path, year, make,lang):
    """
    Saves a dictionary of DataFrames to an Excel file with each DataFrame as a separate sheet.
    
    :param df_dict: Dictionary of DataFrames.
    :param directory_path: Directory path to save the Excel file.
    :param year: Year to be included in the file name.
    :param make: Make to be included in the file name.
    :return: None
    """

    # Create the directory structure if it doesn't exist
    year_dir = os.path.join(directory_path, str(year))
    make_dir = os.path.join(year_dir, make)
    
    os.makedirs(make_dir, exist_ok=True)

    # Create the file name with timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"{year}-{make}_Accessories-UploadFile_{lang}-{timestamp}.xlsx"
    file_path = os.path.join(make_dir, file_name)

    # Save the DataFrames to an Excel file
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:

        combined_sheets_df_by_model = {}

            # First pass: write combined sheets
        for model, trims_dict in dict_all_dfs_by_model_ready.items():
            combined_df = pd.DataFrame()

            for i, (trim, df) in enumerate(trims_dict.items()):
                combined_df = pd.concat([combined_df, df if i == 0 else df.iloc[:]], ignore_index=True)

            combined_sheet_name = f"{model}- Combined"
            combined_df.to_excel(writer, sheet_name=combined_sheet_name, index=False)

            # Store for later use
            combined_sheets_df_by_model[model] = combined_df

        # Second pass: write individual sheets
        for model, trims_dict in dict_all_dfs_by_model_ready.items():
            for trim, df in trims_dict.items():
                sheet_name = f"{model}-{trim}"
                df.to_excel(writer, sheet_name=sheet_name, index=False)

        # Access the workbook and apply green tab color to combined sheets
        workbook = writer.book
        for sheet_name in workbook.sheetnames:
            if "Combined" in sheet_name:
                workbook[sheet_name].sheet_properties.tabColor = "00FF00"  # Bright green


In [27]:
def main(input_master_file_path, destination_path, make, year, lang):
    
    # Reading the master file
    master_dict_dfs, sheets_names = read_excel_sheets(input_master_file_path)

    # Clean dfs
    cleaned_dfs_dic = clean_dataframe_columns(master_dict_dfs)

    # Create unique df for each model
    all_model_df_dict = create_all_unique_model_df(cleaned_dfs_dic, make, year)

    # Save all dfs to excel 
    save_dfs_to_excel(all_model_df_dict, destination_path, year=year, make=make, lang=lang)
    
    print(f"Yeaaah!!!!! Done. Your {lang} file is ready to upload. Check your Directory :)")
    
    

In [21]:
file_name = '2025-Honda-Master-June25'
input_master_file_path = f'C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/landing_zone/2026/2025-06/Honda/{file_name}.xlsx'
processes_files_storage_path ='C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/ready_to_upload'
# file_name ='2025-Honda-Master-June25'

# master_file_path_25 = f'C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/landing_zone/2026/2025-06/Honda/{file_name}.xlsx'

In [40]:
make= "Honda"
input_year = '2026'

Lang = 'FR'

file_name = 'Honda_ACCY_Master_2026_FR'

input_master_file_path = f'C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/landing_zone/2026/Honda/2025-09/{file_name}.xlsx'

main(input_master_file_path= input_master_file_path, destination_path=processes_files_storage_path, make=make, year=input_year, lang =Lang)

C:\Users\paxm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Yeaaah!!!!! Done. Your FR file is ready to upload. Check your Directory :)


In [ ]:
# create a utility that loads a table of vehicle profiles into a csv. This utility will do regular checks to see if there are any new vehicles added to the table and if so, it will update the csv file with the new vehicles. TO make updates, after getting the response from the database, it will check if there is any difference in teh already existing values, in csv catalogue and will update the csv accordingly. Let's say a particular record has had some changes, we'll then do it that way. Make sure that all data are in lower case to make sure that there are no duplicates because of case sensitivity. I want you to create a function, and do not worry about the query, presented as if the query has been emplimentated, and a path where to store the csv will be provided, and this where it will always be.  

def update_vehicle_catalogue(csv_path, new_data):
    """
    Updates the vehicle catalogue CSV with new data from the database.
    
    :param csv_path: Path to the vehicle catalogue CSV file.
    :param new_data: New data from the database as a DataFrame.
    :return: None
    """
    # Load existing catalogue if it exists
    if os.path.exists(csv_path):
        existing_catalogue = pd.read_csv(csv_path)
        existing_catalogue = existing_catalogue.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    else:
        existing_catalogue = pd.DataFrame()

    # Convert new data to lower case for consistency
    new_data = new_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    # Identify new or updated records
    if not existing_catalogue.empty:
        combined = pd.concat([existing_catalogue, new_data]).drop_duplicates(keep=False)
    else:
        combined = new_data

    # If there are changes, update the CSV
    if not combined.empty:
        updated_catalogue = pd.concat([existing_catalogue, combined]).drop_duplicates().reset_index(drop=True)
        updated_catalogue.to_csv(csv_path, index=False)
        print(f"Vehicle catalogue updated at {csv_path}")
    else:
        print("No updates to the vehicle catalogue.")